<a href="https://colab.research.google.com/github/austinbennysmith/CMIP6/blob/main/NAO_prediction_variance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## This notebook will:
* **Plot anomalies of the NAO time series, and running averages of those anomalies (both plotted for each model, averaged across ensemble members)**
* **Plot the standard deviation across ensemble members for each model over time (and include + or - one standard deviation on plots of the anomalies)**
* **Do bootstrapping of the model output and plot 95% Confidence intervals with the anomalies graphs**

In the plots of SSP5-8.5,the y-axis scale is different for different graphs. It appears that there is more of a trend for e.g. CanESM5 than many others, but is this actually true? Or does the scale just make it look that way?

Dimension naming issue with the last models

In [ ]:
## Installs and imports:
!pip install fsspec
!pip install netCDF4
!apt-get -qq install python-cartopy python3-cartopy;
!pip uninstall -y shapely;
!pip install shapely --no-binary shapely;
!pip install eofs
! pip install --upgrade xarray zarr gcsfs cftime nc-time-axis
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

from matplotlib import pyplot as plt
import numpy as np
from numpy import linalg as LA
import pandas as pd
import xarray as xr
import zarr
import fsspec
import gcsfs
from eofs.xarray import Eof
from nc_time_axis import NetCDFTimeConverter, CalendarDateTime
import cftime
from scipy import stats

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.figsize'] = 12, 6

from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample

In [ ]:
df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv') 
df.head()

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,tauv,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
1,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,huss,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
2,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,rlus,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
3,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,rlds,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
4,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,psl,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706


## **List of all experiments in the archive:**

In [ ]:
uexp = []
for i in df.experiment_id:
  if i not in uexp:
    uexp.append(i)
for i in uexp:
  print(i)

histSST
piClim-CH4
piClim-NTCF
piClim-control
ssp370
hist-1950HC
piClim-2xDMS
piClim-2xdust
piClim-2xfire
piClim-2xss
piClim-BC
piClim-HC
piClim-N2O
piClim-OC
piClim-SO2
piClim-aer
hist-piNTCF
hist-piAer
histSST-1950HC
histSST-piAer
histSST-piCH4
histSST-piNTCF
histSST-piO3
piClim-2xNOx
piClim-2xVOC
piClim-NOx
piClim-O3
piClim-VOC
ssp370-lowNTCF
ssp370SST-lowCH4
ssp370SST-lowNTCF
ssp370SST-ssp126Lu
ssp370SST
ssp370pdSST
1pctCO2-bgc
1pctCO2-rad
esm-ssp585
hist-bgc
1pctCO2-cdr
esm-pi-CO2pulse
esm-pi-cdr-pulse
amip-4xCO2
amip-future4K
amip-m4K
amip-p4K
amip
abrupt-2xCO2
abrupt-solp4p
abrupt-0p5xCO2
amip-lwoff
amip-p4K-lwoff
aqua-4xCO2
abrupt-solm4p
aqua-control-lwoff
aqua-control
aqua-p4K-lwoff
aqua-p4K
1pctCO2
abrupt-4xCO2
historical
piControl
esm-hist
esm-piControl
amip-hist
historical-cmip5
piControl-cmip5
esm-piControl-spinup
piControl-spinup
hist-GHG
hist-nat
historical-ext
hist-aer
hist-CO2
hist-GHG-cmip5
hist-aer-cmip5
hist-nat-cmip5
hist-sol
hist-stratO3
hist-totalO3
hist-volc
ssp

In [ ]:
from datetime import date, timedelta
import netCDF4 as nc
# psl_ssp585 = df.query("variable_id == 'psl' & experiment_id == 'ssp585' & institution_id == 'NCAR' & source_id == 'CESM2'")
# psl_ssp585 = df.query("variable_id == 'psl' & experiment_id == 'ssp585' & table_id == 'Amon' & source_id == 'CESM2'")
psl_ssp585 = df.query("variable_id == 'psl' & experiment_id == 'ssp585' & table_id == 'Amon' & source_id == 'CESM2'")
for i in range(len(psl_ssp585)):
  print(i)
print(len(psl_ssp585.zstore.values))

0
1
2
3


Plotting a histogram of the ensemble size for all the models available for the SSP5-8.5 experiment:

In [ ]:
ssp = df.query("variable_id == 'psl' & experiment_id == 'ssp585' & table_id == 'Amon'")
unique_sources = []
histlist = []
for i in ssp['source_id']:
  if i not in unique_sources:
    unique_sources.append(i)
print(unique_sources)

for i in unique_sources:
  x = ssp.query("source_id =="+"'"+i+"'")
  print(str(len(x))+': '+i)
  histlist.append(len(x))
plt.figure(figsize=(10,  10))
bins = np.arange(60)
print(bins)
plt.hist(histlist, bins)
plt.title('Ensemble size histogram')
plt.xlabel('# of Members')
plt.ylabel('Count')

## **Plotting the time series of anomalies averaged across ensemble members for a model, along with a running mean of the anomales:**

(Here I take the running mean of the anomalies and then normalize it, which apparently produces different results than if I normalize the anomalies and then take the running mean...why is this? The next code cell after this one has two extra lines (which are commented out) that do this process in the other order so you can see the difference)

NOTE: I've added a line that selects just the time range from January 1st 2015 to January 1st 2101. For most of the datasets this is unnecessary, but I realized that run 20 of CanESM5 has data through 2300, and I would like to only focus on the 21st century in order to make comparisons to other data.

In [ ]:
### This cell prints a list of the types of datetime data in the archive.
# This is helpful because in the following cell I have a series of if/elif statements that restrict the time range to 2015-2100, since a few of the datasets go longer than this and I am trying to average across time series.
# When running this type of code for different experiments, I need to make sure there are enough if statements in the following cell to account for every type of datetime data in the archive.

def get_types(experiment):
  global uniquetypes
  test = df.query("variable_id == 'psl' & experiment_id == "+"'"+experiment+"'"+" & table_id == 'Amon'")
  usource = []
  for m in test.source_id:
    if m not in usource:
      usource.append(m)

  uniquetypes = []
  for source in usource:
    print(source)
    abc = "variable_id == 'psl' & experiment_id == 'ssp585' & table_id == 'Amon' & source_id == "+"'"+source+"'"
    psl_ssp585 = df.query(abc)
    # print('length: ', len(psl_ssp585.zstore.values))
    for i in range(len(psl_ssp585.zstore.values)):
      zstore = psl_ssp585.zstore.values[i]
      mapper = fsspec.get_mapper(zstore)
      ds = xr.open_zarr(mapper, consolidated=True, decode_times=True) # Make decode_times=True to convert dates to datetime objects?? The problem is, Colab doesn't seem to like my installation of nc-time-axis. I could try a Jupyter Binder, or keep Googling to try to solve this.
      psl = ds.psl.sel(lat=slice(20,80))
      psl = psl.where((ds.lon >= 270) | (ds.lon <= 40), drop=True)
      if type(psl.time.values[0]) not in uniquetypes:
        uniquetypes.append(type(psl.time.values[0]))
      if len(psl.time)!=1032 and len(psl.time)!=1020:
        print('time length:', len(psl.time), 'INDEX:', i)
    # smoothplot()
  print('UNIQUE TYPES: ', uniquetypes)

Plotting the anomalies of the PC time series (and their running average) and the same for the standard deviation across ensemble members of the anomalies

* Standard deviation is consistently higher in the Winter than the summer because there are more storms in the Winter, which causes more short-term variability in the NAOI

In [ ]:
def smoothplot():
  global psl
  global eigenpatterns
  global meanpatterns
  eigenpatterns = {}
  SEASONS = ['DJF', 'MAM', 'JJA', 'SON']
  # Moving average code from https://stackoverflow.com/questions/14313510/how-to-calculate-rolling-moving-average-using-numpy-scipy
  def moving_average(x, w):
      return np.convolve(x, np.ones(w), 'valid') / w
  eigenpatterns['singVal 0']={
      'anomalies': {},
  }
  for key in eigenpatterns['singVal 0']:
    for season in SEASONS:
      eigenpatterns['singVal 0'][key][season] = []
  for member in range(len(psl_ssp585.zstore.values)): 
    print(member)
    # Accessing the file, getting it to just the lats and lons I want:
    zstore = psl_ssp585.zstore.values[member]
    mapper = fsspec.get_mapper(zstore)
    ds = xr.open_zarr(mapper, consolidated=True, decode_times=True) # Make decode_times=True to convert dates to datetime objects?? The problem is, Colab doesn't seem to like my installation of nc-time-axis. I could try a Jupyter Binder, or keep Googling to try to solve this.
    psl = ds.psl.sel(lat=slice(20,80))
    psl = psl.where((ds.lon >= 270) | (ds.lon <= 40), drop=True)
    lat = ds.lat.sel(lat=slice(20,80))
    lon = ds.lon.where((ds.lon >= 270) | (ds.lon <= 40), drop=True) # Not sure exactly what drop means, but I think it doesn't matter
    if isinstance((psl.time.values[member]), np.datetime64):
      psl = psl.sel(time=slice('2015-01-01T12:00:00.000000000', '2101-01-01T12:00:00.000000000'))
    elif isinstance((psl.time.values[member]), cftime._cftime.DatetimeNoLeap):
      psl = psl.sel(time=slice(cftime.DatetimeNoLeap(2015, 1, 1, 12, 0, 0, 0), cftime.DatetimeNoLeap(2101, 1, 1, 12, 0, 0, 0)))
    elif isinstance((psl.time.values[member]), cftime._cftime.DatetimeJulian):
      psl = psl.sel(time=slice(cftime.DatetimeJulian(2015, 1, 1, 12, 0, 0, 0), cftime.DatetimeJulian(2101, 1, 1, 12, 0, 0, 0)))
    elif isinstance((psl.time.values[member]), cftime._cftime.Datetime360Day):
      psl = psl.sel(time=slice(cftime.Datetime360Day(2015, 1, 1, 12, 0, 0, 0), cftime.Datetime360Day(2101, 1, 1, 12, 0, 0, 0)))


    # Useful stackexchange post: https://stackoverflow.com/questions/40272222/select-xarray-pandas-index-based-on-specific-months
    def is_djf(month):
      return (month>=12) | ((month>=1) & (month<=2))
    def is_mam(month):
      return (month>=3) & (month<=5)
    def is_jja(month):
      return (month>=6) & (month<=8)
    def is_son(month):
      return (month>=9) & (month<=11)
    psl_djf = psl.sel(time=is_djf(psl['time.month']))
    psl_mam = psl.sel(time=is_mam(psl['time.month']))
    psl_jja = psl.sel(time=is_jja(psl['time.month']))
    psl_son = psl.sel(time=is_son(psl['time.month']))


    psl_allseasons = {
        'DJF': psl_djf,
        'MAM': psl_mam,
        'JJA': psl_jja,
        'SON': psl_son
    }
    century_trends = {
        'DJF': {},
        'JJA': {},
        'MAM': {},
        'SON': {}
    }
    for key in psl_allseasons.keys():
      tlength = len(psl_allseasons[key].time) # The psl_WEIGHTED dictionary does not contain DataArray objects, so I can't just get the time from that.
      latlength = len(psl_allseasons[key].lat)
      lonlength = len(psl_allseasons[key].lon)
      psl2d = np.reshape(psl_allseasons[key].values, (tlength, latlength*lonlength)) # This is the only line in the for loop I changed when adding in the weighting
      psl2d = np.matrix.transpose(psl2d)
      Unow, snow, VTnow = LA.svd(psl2d, full_matrices=False)
      century_trends[key]['U'] = Unow
      century_trends[key]['s'] = snow
      century_trends[key]['VT'] = VTnow
    for season in SEASONS:
      eigenpatterns['singVal 0']['anomalies'][season].append(century_trends[season]['VT'][0]-np.mean(century_trends[season]['VT'][0]))
    
  meanpatterns = {}
  for singVal in eigenpatterns:
    meanpatterns[singVal] = {}
    meanpatterns[singVal]['std'] = {}
    for kind in eigenpatterns[singVal]:
      meanpatterns[singVal][kind] = {}
      for season in eigenpatterns[singVal][kind]:
        stacked = np.stack([i for i in eigenpatterns[singVal][kind][season]], axis=0)
        meanpatterns[singVal][kind][season] = stacked.mean(axis=0)
        meanpatterns[singVal]['std'][season] = stacked.std(axis=0)
  meanpatterns['singVal 0']['smoothed'] = {}
  for season in SEASONS:
    nao_smoothed = moving_average(meanpatterns['singVal 0']['anomalies'][season], 30)
    meanpatterns['singVal 0']['smoothed'][season] = nao_smoothed
    
  # for member in range(len(psl_ssp585.zstore.values)):
      # nao_smoothed = moving_average(eigenpatterns['singVal 0']['anomalies'][season][member], 30) # Doing the moving average for 30 points because each season is 3 months per year - so this is a 10 year running mean
      # eigenpatterns['singVal 0']['smoothed'][season].append(nao_smoothed/np.std(nao_smoothed))
    
  fig, axs = plt.subplots(2, 2, figsize=[24, 12])
  axlist = axs.flatten()
  for season in SEASONS:
    # x = np.arange(0, 495)
    x = np.arange(0, len(nao_smoothed))
    axlist[SEASONS.index(season)].fill_between(x, 0, meanpatterns['singVal 0']['smoothed'][season], where=meanpatterns['singVal 0']['smoothed'][season]>0, color='r', label='10 year running mean')
    axlist[SEASONS.index(season)].fill_between(x, 0, meanpatterns['singVal 0']['smoothed'][season], where=meanpatterns['singVal 0']['smoothed'][season]<0, color='b', label='10 year running mean')
    axlist[SEASONS.index(season)].plot(meanpatterns['singVal 0']['anomalies'][season], 'silver', zorder=0, label='Unsmoothed')
    # plt.plot(x)
    axlist[SEASONS.index(season)].set_title('NAO Index (PC 1) '+season)
    axlist[SEASONS.index(season)].set_ylabel('Right Singular Vector Anomalies')
    axlist[SEASONS.index(season)].set_xlabel('Time')
    axlist[SEASONS.index(season)].legend()
  plt.show()
  fig, axs = plt.subplots(2, 2, figsize=[24, 12])
  axlist = axs.flatten()
  for season in SEASONS:
    # x = np.arange(0, 495)
    x = np.arange(0, len(nao_smoothed))
    axlist[SEASONS.index(season)].plot(meanpatterns['singVal 0']['std'][season], 'silver', zorder=0, label='Unsmoothed')
    axlist[SEASONS.index(season)].plot(moving_average(meanpatterns['singVal 0']['std'][season], 30), 'red', label='Smoothed')
    # plt.plot(x)
    axlist[SEASONS.index(season)].set_title('Prediction standard deviation '+season)
    axlist[SEASONS.index(season)].set_ylabel('Standard deviation across ensemble members')
    axlist[SEASONS.index(season)].set_xlabel('Time')
    axlist[SEASONS.index(season)].legend()
  plt.show()
smoothplot()

0
1
2
3
4
5
6
7
8
9
10


KeyboardInterrupt: ignored

Making the aforementioned plots for every model that ran the SSP5-8.5 experiment

In [ ]:
test = df.query("variable_id == 'psl' & experiment_id == 'ssp585' & table_id == 'Amon'")
usource = []
for m in test.source_id:
  if m not in usource:
    usource.append(m)

for source in usource:
  print(source)
  abc = "variable_id == 'psl' & experiment_id == 'ssp585' & table_id == 'Amon' & source_id == "+"'"+source+"'"
  psl_ssp585 = df.query(abc)
  smoothplot()

## **Same plots as before, but with the averages plus or minus the standard deviation:**

In [ ]:
def smoothplot():
  global psl
  global eigenpatterns
  global meanpatterns
  eigenpatterns = {}
  SEASONS = ['DJF', 'MAM', 'JJA', 'SON']
  # Moving average code from https://stackoverflow.com/questions/14313510/how-to-calculate-rolling-moving-average-using-numpy-scipy
  def moving_average(x, w):
      return np.convolve(x, np.ones(w), 'valid') / w
  eigenpatterns['singVal 0']={
      'anomalies': {},
  }
  for key in eigenpatterns['singVal 0']:
    for season in SEASONS:
      eigenpatterns['singVal 0'][key][season] = []
  for member in range(len(psl_ssp585.zstore.values)): 
    print(member)
    # Accessing the file, getting it to just the lats and lons I want:
    zstore = psl_ssp585.zstore.values[member]
    mapper = fsspec.get_mapper(zstore)
    ds = xr.open_zarr(mapper, consolidated=True, decode_times=True) # Make decode_times=True to convert dates to datetime objects?? The problem is, Colab doesn't seem to like my installation of nc-time-axis. I could try a Jupyter Binder, or keep Googling to try to solve this.
    psl = ds.psl.sel(lat=slice(20,80))
    psl = psl.where((ds.lon >= 270) | (ds.lon <= 40), drop=True)
    lat = ds.lat.sel(lat=slice(20,80))
    lon = ds.lon.where((ds.lon >= 270) | (ds.lon <= 40), drop=True) # Not sure exactly what drop means, but I think it doesn't matter
    if isinstance((psl.time.values[member]), np.datetime64):
      psl = psl.sel(time=slice('2015-01-01T12:00:00.000000000', '2101-01-01T12:00:00.000000000'))
    elif isinstance((psl.time.values[member]), cftime._cftime.DatetimeNoLeap):
      psl = psl.sel(time=slice(cftime.DatetimeNoLeap(2015, 1, 1, 12, 0, 0, 0), cftime.DatetimeNoLeap(2101, 1, 1, 12, 0, 0, 0)))
    elif isinstance((psl.time.values[member]), cftime._cftime.DatetimeJulian):
      psl = psl.sel(time=slice(cftime.DatetimeJulian(2015, 1, 1, 12, 0, 0, 0), cftime.DatetimeJulian(2101, 1, 1, 12, 0, 0, 0)))
    elif isinstance((psl.time.values[member]), cftime._cftime.Datetime360Day):
      psl = psl.sel(time=slice(cftime.Datetime360Day(2015, 1, 1, 12, 0, 0, 0), cftime.Datetime360Day(2101, 1, 1, 12, 0, 0, 0)))


    # Useful stackexchange post: https://stackoverflow.com/questions/40272222/select-xarray-pandas-index-based-on-specific-months
    def is_djf(month):
      return (month>=12) | ((month>=1) & (month<=2))
    def is_mam(month):
      return (month>=3) & (month<=5)
    def is_jja(month):
      return (month>=6) & (month<=8)
    def is_son(month):
      return (month>=9) & (month<=11)
    psl_djf = psl.sel(time=is_djf(psl['time.month']))
    psl_mam = psl.sel(time=is_mam(psl['time.month']))
    psl_jja = psl.sel(time=is_jja(psl['time.month']))
    psl_son = psl.sel(time=is_son(psl['time.month']))


    psl_allseasons = {
        'DJF': psl_djf,
        'MAM': psl_mam,
        'JJA': psl_jja,
        'SON': psl_son
    }
    century_trends = {
        'DJF': {},
        'JJA': {},
        'MAM': {},
        'SON': {}
    }
    for key in psl_allseasons.keys():
      tlength = len(psl_allseasons[key].time) # The psl_WEIGHTED dictionary does not contain DataArray objects, so I can't just get the time from that.
      latlength = len(psl_allseasons[key].lat)
      lonlength = len(psl_allseasons[key].lon)
      psl2d = np.reshape(psl_allseasons[key].values, (tlength, latlength*lonlength)) # This is the only line in the for loop I changed when adding in the weighting
      psl2d = np.matrix.transpose(psl2d)
      Unow, snow, VTnow = LA.svd(psl2d, full_matrices=False)
      century_trends[key]['U'] = Unow
      century_trends[key]['s'] = snow
      century_trends[key]['VT'] = VTnow
    for season in SEASONS:
      eigenpatterns['singVal 0']['anomalies'][season].append(century_trends[season]['VT'][0]-np.mean(century_trends[season]['VT'][0]))
    
  meanpatterns = {}
  for singVal in eigenpatterns:
    meanpatterns[singVal] = {}
    meanpatterns[singVal]['std'] = {}
    for kind in eigenpatterns[singVal]:
      meanpatterns[singVal][kind] = {}
      for season in eigenpatterns[singVal][kind]:
        stacked = np.stack([i for i in eigenpatterns[singVal][kind][season]], axis=0)
        meanpatterns[singVal][kind][season] = stacked.mean(axis=0)
        meanpatterns[singVal]['std'][season] = stacked.std(axis=0)
  meanpatterns['singVal 0']['smoothed'] = {}
  for season in SEASONS:
    nao_smoothed = moving_average(meanpatterns['singVal 0']['anomalies'][season], 30)
    meanpatterns['singVal 0']['smoothed'][season] = nao_smoothed
    
  # for member in range(len(psl_ssp585.zstore.values)):
      # nao_smoothed = moving_average(eigenpatterns['singVal 0']['anomalies'][season][member], 30) # Doing the moving average for 30 points because each season is 3 months per year - so this is a 10 year running mean
      # eigenpatterns['singVal 0']['smoothed'][season].append(nao_smoothed/np.std(nao_smoothed))
    
  fig, axs = plt.subplots(2, 2, figsize=[24, 12])
  axlist = axs.flatten()
  for season in SEASONS:
    # x = np.arange(0, 495)
    x = np.arange(0, len(nao_smoothed))
    std_below = meanpatterns['singVal 0']['smoothed'][season]-moving_average(meanpatterns['singVal 0']['std'][season], 30)
    std_above = meanpatterns['singVal 0']['smoothed'][season]+moving_average(meanpatterns['singVal 0']['std'][season], 30)
    axlist[SEASONS.index(season)].fill_between(x, 0, meanpatterns['singVal 0']['smoothed'][season], where=meanpatterns['singVal 0']['smoothed'][season]>0, color='r', label='10 year running mean')
    axlist[SEASONS.index(season)].fill_between(x, 0, meanpatterns['singVal 0']['smoothed'][season], where=meanpatterns['singVal 0']['smoothed'][season]<0, color='b', label='10 year running mean')
    # axlist[SEASONS.index(season)].plot(meanpatterns['singVal 0']['smoothed'][season], color='k', label = '10 year running mean')
    axlist[SEASONS.index(season)].plot(std_below, color='g')
    axlist[SEASONS.index(season)].plot(std_above, color = 'g')
    axlist[SEASONS.index(season)].fill_between(x, std_below, std_above, color='g', alpha = 0.3)
    axlist[SEASONS.index(season)].plot(meanpatterns['singVal 0']['anomalies'][season], 'silver', zorder=0, label='Unsmoothed')
    # plt.plot(x)
    axlist[SEASONS.index(season)].set_title('NAO Index (PC 1) '+season)
    axlist[SEASONS.index(season)].set_ylabel('Right Singular Vector Anomalies')
    axlist[SEASONS.index(season)].set_xlabel('Time')
    axlist[SEASONS.index(season)].legend()
  plt.show()
  fig, axs = plt.subplots(2, 2, figsize=[24, 12])
  axlist = axs.flatten()
  for season in SEASONS:
    # x = np.arange(0, 495)
    x = np.arange(0, len(nao_smoothed))
    axlist[SEASONS.index(season)].plot(meanpatterns['singVal 0']['std'][season], 'silver', zorder=0, label='Unsmoothed')
    axlist[SEASONS.index(season)].plot(moving_average(meanpatterns['singVal 0']['std'][season], 30), 'red', label='Smoothed')
    # plt.plot(x)
    axlist[SEASONS.index(season)].set_title('Prediction standard deviation '+season)
    axlist[SEASONS.index(season)].set_ylabel('Standard deviation across ensemble members')
    axlist[SEASONS.index(season)].set_xlabel('Time')
    axlist[SEASONS.index(season)].legend()
  plt.show()
smoothplot()

**Adding in Confidence intervals:**

(as can be seen in the graphs later on, this is not completely working yet (the confidence intervals of the standard deviation do not always include the standard deviation graph))



In [ ]:
def smoothplot():
  global psl
  global eigenpatterns
  global meanpatterns
  eigenpatterns = {}
  SEASONS = ['DJF', 'MAM', 'JJA', 'SON']
  # Moving average code from https://stackoverflow.com/questions/14313510/how-to-calculate-rolling-moving-average-using-numpy-scipy
  def moving_average(x, w):
      return np.convolve(x, np.ones(w), 'valid') / w
  eigenpatterns['singVal 0']={
      'anomalies': {},
  }
  for key in eigenpatterns['singVal 0']:
    for season in SEASONS:
      eigenpatterns['singVal 0'][key][season] = []
  for member in range(len(psl_ssp585.zstore.values)): 
    print(member)
    # Accessing the file, getting it to just the lats and lons I want:
    zstore = psl_ssp585.zstore.values[member]
    mapper = fsspec.get_mapper(zstore)
    ds = xr.open_zarr(mapper, consolidated=True, decode_times=True) # Make decode_times=True to convert dates to datetime objects?? The problem is, Colab doesn't seem to like my installation of nc-time-axis. I could try a Jupyter Binder, or keep Googling to try to solve this.
    psl = ds.psl.sel(lat=slice(20,80))
    psl = psl.where((ds.lon >= 270) | (ds.lon <= 40), drop=True)
    lat = ds.lat.sel(lat=slice(20,80))
    lon = ds.lon.where((ds.lon >= 270) | (ds.lon <= 40), drop=True) # Not sure exactly what drop means, but I think it doesn't matter
    if isinstance((psl.time.values[member]), np.datetime64):
      psl = psl.sel(time=slice('2015-01-01T12:00:00.000000000', '2101-01-01T12:00:00.000000000'))
    elif isinstance((psl.time.values[member]), cftime._cftime.DatetimeNoLeap):
      psl = psl.sel(time=slice(cftime.DatetimeNoLeap(2015, 1, 1, 12, 0, 0, 0), cftime.DatetimeNoLeap(2101, 1, 1, 12, 0, 0, 0)))
    elif isinstance((psl.time.values[member]), cftime._cftime.DatetimeJulian):
      psl = psl.sel(time=slice(cftime.DatetimeJulian(2015, 1, 1, 12, 0, 0, 0), cftime.DatetimeJulian(2101, 1, 1, 12, 0, 0, 0)))
    elif isinstance((psl.time.values[member]), cftime._cftime.Datetime360Day):
      psl = psl.sel(time=slice(cftime.Datetime360Day(2015, 1, 1, 12, 0, 0, 0), cftime.Datetime360Day(2101, 1, 1, 12, 0, 0, 0)))


    # Useful stackexchange post: https://stackoverflow.com/questions/40272222/select-xarray-pandas-index-based-on-specific-months
    def is_djf(month):
      return (month>=12) | ((month>=1) & (month<=2))
    def is_mam(month):
      return (month>=3) & (month<=5)
    def is_jja(month):
      return (month>=6) & (month<=8)
    def is_son(month):
      return (month>=9) & (month<=11)
    psl_djf = psl.sel(time=is_djf(psl['time.month']))
    psl_mam = psl.sel(time=is_mam(psl['time.month']))
    psl_jja = psl.sel(time=is_jja(psl['time.month']))
    psl_son = psl.sel(time=is_son(psl['time.month']))


    psl_allseasons = {
        'DJF': psl_djf,
        'MAM': psl_mam,
        'JJA': psl_jja,
        'SON': psl_son
    }
    century_trends = {
        'DJF': {},
        'JJA': {},
        'MAM': {},
        'SON': {}
    }
    for key in psl_allseasons.keys():
      tlength = len(psl_allseasons[key].time) # The psl_WEIGHTED dictionary does not contain DataArray objects, so I can't just get the time from that.
      latlength = len(psl_allseasons[key].lat)
      lonlength = len(psl_allseasons[key].lon)
      psl2d = np.reshape(psl_allseasons[key].values, (tlength, latlength*lonlength)) # This is the only line in the for loop I changed when adding in the weighting
      psl2d = np.matrix.transpose(psl2d)
      Unow, snow, VTnow = LA.svd(psl2d, full_matrices=False)
      century_trends[key]['U'] = Unow
      century_trends[key]['s'] = snow
      century_trends[key]['VT'] = VTnow
    for season in SEASONS:
      eigenpatterns['singVal 0']['anomalies'][season].append(century_trends[season]['VT'][0]-np.mean(century_trends[season]['VT'][0]))
    
  meanpatterns = {}
  for singVal in eigenpatterns:
    meanpatterns[singVal] = {}
    meanpatterns[singVal]['std'] = {}
    for kind in eigenpatterns[singVal]:
      meanpatterns[singVal][kind] = {}
      meanpatterns[singVal]['stacked'] = {}
      ## The following seems to be unnecessary:
      meanpatterns[singVal]['bootstrapping std'] = {}
      for season in eigenpatterns[singVal][kind]:
        stacked = np.stack([i for i in eigenpatterns[singVal][kind][season]], axis=0)
        meanpatterns[singVal][kind][season] = stacked.mean(axis=0)
        meanpatterns[singVal]['std'][season] = stacked.std(axis=0)
        # stacked = np.stack([i for i in eigenpatterns[singVal][kind][season]], axis=0)
        meanpatterns[singVal]['stacked'][season] = stacked
        sample_hist = []

  meanpatterns['singVal 0']['smoothed'] = {}
  for season in SEASONS:
    nao_smoothed = moving_average(meanpatterns['singVal 0']['anomalies'][season], 30)
    meanpatterns['singVal 0']['smoothed'][season] = nao_smoothed
  
  Sboot = {
      'DJF': [],
      'JJA': [],
      'SON': [],
      'MAM': []
  }
  fig, axs = plt.subplots(2, 2, figsize=[24, 12])
  axlist = axs.flatten()
  for season in SEASONS:
    ## Loading in the data (a stacked ndarray of NAOI time series):
    data = meanpatterns[singVal]['stacked'][season]
    ## Setting up indices to be resampled. len(data) is the number of ensemble members, and so if there are 50 members the indices I'm resampling should be 0 through 50. Thus if I resample 1000 times I get 1000 different stacked ndarrays, each with a randomly chosen set of 50 members (chosen with replacement)
    indices = np.arange(len(data))

    ## I will include the means of the resampled data as well as the standard deviations
    resample_dict = {
        'resample means': {},
        'resample std': {},
    }
    for i in range(1000):
      index_resample = resample(indices, replace=True, n_samples=len(data))
      # print(index_resample)
      resample_stack = np.stack([data[index_resample[j]] for j in range(len(index_resample))], axis=0)
      resample_means = np.mean(resample_stack, axis=0)
      resample_dict['resample means'].append(resample_means)
      resample_std = np.std(resample_stack, axis=0)
      resample_dict['resample std'].append(resample_std)

    resample_dict['resample means'] = np.stack(resample_dict['resample means'], axis=0)
    bootmean = resample_dict['resample means'].mean(axis=0)
    boot_std = resample_dict['resample means'].std(axis=0)
    bootmean_smoothed = moving_average(bootmean, 30)

    resample_dict['resample std'] = np.stack(resample_dict['resample std'], axis=0)
    Sbootmean = resample_dict['resample std'].mean(axis=0)
    Sboot_std = resample_dict['resample std'].std(axis=0)
    Sbootmean_smoothed = moving_average(Sbootmean, 30)

    # What's the best point at which to do the moving average?
    # Here I'm adding/subtracting the mean to/from the standard deviation, then taking the moving average. Is that correct? Or should I do that in reverse order? Does it matter?
    boot_below = bootmean-2*boot_std
    boot_above = bootmean+2*boot_std
    boot_below_smoothed = moving_average(boot_below, 30)
    boot_above_smoothed = moving_average(boot_above, 30)
    x = np.arange(len(boot_below_smoothed))

    Sboot_below = Sbootmean-2*Sboot_std
    Sboot_above = Sbootmean+2*Sboot_std
    Sboot_below_smoothed = moving_average(Sboot_below, 30)
    Sboot_above_smoothed = moving_average(Sboot_above, 30)
    Sboot[season]['below'] = Sboot_below_smoothed
    Sboot[season]['above'] = Sboot_above_smoothed

    # axlist[SEASONS.index(season)].plot(moving_average(data.mean(axis=0), 30), 'k', lw=5, zorder=3)
    axlist[SEASONS.index(season)].plot(moving_average(meanpatterns['singVal 0']['anomalies'][season], 30), 'k', lw=5, zorder=3)
    axlist[SEASONS.index(season)].plot(moving_average(boot_below, 30), 'g', lw=3, label='95% CI')
    axlist[SEASONS.index(season)].plot(moving_average(boot_above, 30), 'g', lw=3, label='95% CI')
    axlist[SEASONS.index(season)].fill_between(x, 0, moving_average(data.mean(axis=0), 30), where=moving_average(data.mean(axis=0), 30)>0, color='r', label='10 year running mean')
    axlist[SEASONS.index(season)].fill_between(x, 0, moving_average(data.mean(axis=0), 30), moving_average(data.mean(axis=0), 30)<0, color='b', label='10 year running mean')
    axlist[SEASONS.index(season)].fill_between(x, boot_below_smoothed, boot_above_smoothed, color='g', alpha=0.3)
    axlist[SEASONS.index(season)].legend()
  plt.show()



  # fig, axs = plt.subplots(2, 2, figsize=[24, 12])
  # axlist = axs.flatten()
  # for season in SEASONS:
  #   # x = np.arange(0, 495)
  #   x = np.arange(0, len(nao_smoothed))
  #   axlist[SEASONS.index(season)].plot(meanpatterns['singVal 0']['std'][season], 'silver', zorder=0, label='Unsmoothed')
  #   axlist[SEASONS.index(season)].plot(moving_average(meanpatterns['singVal 0']['std'][season], 30), 'k', lw=5, label='Smoothed')
  #   axlist[SEASONS.index(season)].plot(moving_average(Sboot_below, 30), 'g', lw=3, label='95% CI')
  #   axlist[SEASONS.index(season)].plot(moving_average(Sboot_above, 30), 'g', lw=3, label='95% CI')
  #   axlist[SEASONS.index(season)].fill_between(x, Sboot_below_smoothed, Sboot_above_smoothed, color='g', alpha=0.3)
  #   # plt.plot(x)
  #   axlist[SEASONS.index(season)].set_title('Prediction standard deviation '+season)
  #   axlist[SEASONS.index(season)].set_ylabel('Standard deviation across ensemble members')
  #   axlist[SEASONS.index(season)].set_xlabel('Time')
  #   axlist[SEASONS.index(season)].legend()
  # plt.show()
  fig, axs = plt.subplots(2, 2, figsize=[24, 12])
  axlist = axs.flatten()
  for season in SEASONS:
    # x = np.arange(0, 495)
    x = np.arange(0, len(nao_smoothed))
    axlist[SEASONS.index(season)].plot(meanpatterns['singVal 0']['std'][season], 'silver', zorder=0, label='Unsmoothed')
    axlist[SEASONS.index(season)].plot(moving_average(meanpatterns['singVal 0']['std'][season], 30), 'k', lw=5, label='Smoothed')
    axlist[SEASONS.index(season)].plot(Sboot[season]['below'], 'g', lw=3, label='95% CI')
    axlist[SEASONS.index(season)].plot(Sboot[season]['above'], 'g', lw=3, label='95% CI')
    axlist[SEASONS.index(season)].fill_between(x, Sboot[season]['below'], Sboot[season]['above'], color='g', alpha=0.3)
    # plt.plot(x)
    axlist[SEASONS.index(season)].set_title('Prediction standard deviation '+season)
    axlist[SEASONS.index(season)].set_ylabel('Standard deviation across ensemble members')
    axlist[SEASONS.index(season)].set_xlabel('Time')
    axlist[SEASONS.index(season)].legend()
  plt.show()
smoothplot()

The following cell does the plots of the anomalies & standard deviation & confidence intervals for a single dataset rather than all of them (this is just useful for debugging...and there does seem to still be a bug)

NEXT:

* Make sure everything plotted (2 cells down) the way I expected (without the one that used to be the last one) check

* Try out the cell below, in which I implemented confidence intervals for the resampled standard deviation. Debug.

* TEST FOR SIZE OF THE ARRAY TO MAKE SURE IT HAS 1000 ROWS

* Add in unsmoothed data in light grey in the background of the NAOI mean plot??

In [ ]:
## Randomly choose a set of ensemble members with replacement, then get the mean of that randomly chosen sample and append the mean to a list

def moving_average(x, w):
  return np.convolve(x, np.ones(w), 'valid') / w

SEASONS = ['DJF', 'MAM', 'JJA', 'SON']
fig, axs = plt.subplots(2, 2, figsize=[24, 12])
axlist = axs.flatten()
for season in SEASONS:
  data = meanpatterns[singVal]['stacked'][season]
  # data[0].shape
  indices = np.arange(len(data))

  resample_dict = {
      'resample means': [],
      'resample std': [],
  }
  for i in range(1000):
    index_resample = resample(indices, replace=True, n_samples=len(data))
    # print(index_resample)
    resample_stack = np.stack([data[index_resample[j]] for j in range(len(index_resample))], axis=0)
    resample_means = np.mean(resample_stack, axis=0)
    resample_stds = np.std(resample_stack, axis=0)
    resample_dict['resample means'].append(resample_means)
    resample_dict['resample std'].append(resample_stds)
    # resample_std = np.std(resample_stack, axis=0)

  resample_dict['resample means'] = np.stack(resample_dict['resample means'], axis=0)
  bootmean = resample_dict['resample means'].mean(axis=0)
  bootmean_std = resample_dict['resample means'].std(axis=0)
  bootmean_smoothed = moving_average(bootmean, 30)

  resample_dict['resample std'] = np.stack(resample_dict['resample std'], axis=0)
  bootstd = resample_dict['resample std'].mean(axis=0)
  bootstd_std = resample_dict['resample std'].std(axis=0)
  bootstd_smoothed = moving_average(bootstd, 30)
  

  # What's the best point at which to do the moving average?
  # Here I'm adding/subtracting the mean to/from the standard deviation, then taking the moving average. Is that correct? Or should I do that in reverse order? Does it matter?
  bootmean_below = bootmean-2*bootmean_std
  bootmean_above = bootmean+2*bootmean_std
  bootmean_below_smoothed = moving_average(bootmean_below, 30)
  bootmean_above_smoothed = moving_average(bootmean_above, 30)
  x = np.arange(len(bootmean_below_smoothed))

  bootstd_below = bootstd-2*bootstd_std
  bootstd_above = bootstd+2*bootstd_std
  bootstd_below_smoothed = moving_average(bootstd_below, 30)
  bootstd_above_smoothed = moving_average(bootstd_above, 30)

  axlist[SEASONS.index(season)].plot(moving_average(data.mean(axis=0), 30), 'k', lw=5, zorder=3)
  axlist[SEASONS.index(season)].plot(moving_average(bootmean_below, 30), 'g', lw=3, label='95% CI')
  axlist[SEASONS.index(season)].plot(moving_average(bootmean_above, 30), 'g', lw=3, label='95% CI')
  axlist[SEASONS.index(season)].fill_between(x, 0, moving_average(data.mean(axis=0), 30), where=moving_average(data.mean(axis=0), 30)>0, color='r', label='10 year running mean')
  axlist[SEASONS.index(season)].fill_between(x, 0, moving_average(data.mean(axis=0), 30), moving_average(data.mean(axis=0), 30)<0, color='b', label='10 year running mean')
  axlist[SEASONS.index(season)].fill_between(x, bootmean_below_smoothed, bootmean_above_smoothed, color='g', alpha=0.3)
  axlist[SEASONS.index(season)].legend()
plt.show()
fig, axs = plt.subplots(2, 2, figsize=[24, 12])
axlist = axs.flatten()
for season in SEASONS:
  axlist[SEASONS.index(season)].plot(moving_average(data.std(axis=0), 30), 'red', label='Smoothed')
  axlist[SEASONS.index(season)].plot(data.std(axis=0), 'silver', zorder=0, label='Unsmoothed')
  axlist[SEASONS.index(season)].fill_between(x, bootstd_below_smoothed, bootstd_above_smoothed, color='g', alpha=0.3)
  # axlist[SEASONS.index(season)].plot(meanpatterns['singVal 0']['std'][season], 'silver', zorder=0, label='Unsmoothed')
  # axlist[SEASONS.index(season)].plot(moving_average(meanpatterns['singVal 0']['std'][season], 30), 'red', label='Smoothed')
  # axlist[SEASONS.index(season)].set_title('Prediction standard deviation '+season)
  # axlist[SEASONS.index(season)].set_ylabel('Standard deviation across ensemble members')
  # axlist[SEASONS.index(season)].set_xlabel('Time')
  axlist[SEASONS.index(season)].legend()

Making the confidence intervals plots for every model that ran the SSP5-8.5 experiment:

In [ ]:
test = df.query("variable_id == 'psl' & experiment_id == 'ssp585' & table_id == 'Amon'")
usource = []
for m in test.source_id:
  if m not in usource:
    usource.append(m)

for source in usource:
  print(source)
  if source == 'MCM-UA-1-0': # This model has different naming for its dimensions, making it annoying. I might fix this later (although this one only has one ensemble member so it may not be useful anyway)
    continue
  abc = "variable_id == 'psl' & experiment_id == 'ssp585' & table_id == 'Amon' & source_id == "+"'"+source+"'"
  psl_ssp585 = df.query(abc)
  smoothplot()

For the SSP5-3.4 experiment:

In [ ]:
test = df.query("variable_id == 'psl' & experiment_id == 'ssp534' & table_id == 'Amon'")
usource = []
for m in test.source_id:
  if m not in usource:
    usource.append(m)

for source in usource:
  print(source)
  if source == 'MCM-UA-1-0': # This model has different naming for its dimensions, making it annoying. I might fix this later (although this one only has one ensemble member so it may not be useful anyway)
    continue
  abc = "variable_id == 'psl' & experiment_id == 'ssp534' & table_id == 'Amon' & source_id == "+"'"+source+"'"
  psl_ssp585 = df.query(abc)
  smoothplot()

## **The following cell does the plotting for each individual member instead of averaging, with similar architecture to the previous cells**

In [ ]:
def smoothplot():
  global psl
  global eigenpatterns
  eigenpatterns = {}
  SEASONS = ['DJF', 'MAM', 'JJA', 'SON']
  # Moving average code from https://stackoverflow.com/questions/14313510/how-to-calculate-rolling-moving-average-using-numpy-scipy
  def moving_average(x, w):
      return np.convolve(x, np.ones(w), 'valid') / w
  eigenpatterns['singVal 0']={
      'anomalies': {},
      'normalized': {},
      'smoothed': {}
  }
  for key in eigenpatterns['singVal 0']:
    for season in SEASONS:
      eigenpatterns['singVal 0'][key][season] = []
  for member in range(len(psl_ssp585.zstore.values)): 
    print(member)
    # Accessing the file, getting it to just the lats and lons I want:
    zstore = psl_ssp585.zstore.values[member]
    mapper = fsspec.get_mapper(zstore)
    ds = xr.open_zarr(mapper, consolidated=True, decode_times=True) # Make decode_times=True to convert dates to datetime objects?? The problem is, Colab doesn't seem to like my installation of nc-time-axis. I could try a Jupyter Binder, or keep Googling to try to solve this.
    psl = ds.psl.sel(lat=slice(20,80))
    psl = psl.where((ds.lon >= 270) | (ds.lon <= 40), drop=True)
    lat = ds.lat.sel(lat=slice(20,80))
    lon = ds.lon.where((ds.lon >= 270) | (ds.lon <= 40), drop=True) # Not sure exactly what drop means, but I think it doesn't matter


    # Useful stackexchange post: https://stackoverflow.com/questions/40272222/select-xarray-pandas-index-based-on-specific-months
    def is_djf(month):
      return (month>=12) | ((month>=1) & (month<=2))
    def is_mam(month):
      return (month>=3) & (month<=5)
    def is_jja(month):
      return (month>=6) & (month<=8)
    def is_son(month):
      return (month>=9) & (month<=11)
    psl_djf = psl.sel(time=is_djf(psl['time.month']))
    psl_mam = psl.sel(time=is_mam(psl['time.month']))
    psl_jja = psl.sel(time=is_jja(psl['time.month']))
    psl_son = psl.sel(time=is_son(psl['time.month']))


    psl_allseasons = {
        'DJF': psl_djf,
        'MAM': psl_mam,
        'JJA': psl_jja,
        'SON': psl_son
    }
    century_trends = {
        'DJF': {},
        'JJA': {},
        'MAM': {},
        'SON': {}
    }
    for key in psl_allseasons.keys():
      tlength = len(psl_allseasons[key].time) # The psl_WEIGHTED dictionary does not contain DataArray objects, so I can't just get the time from that.
      latlength = len(psl_allseasons[key].lat)
      lonlength = len(psl_allseasons[key].lon)
      psl2d = np.reshape(psl_allseasons[key].values, (tlength, latlength*lonlength)) # This is the only line in the for loop I changed when adding in the weighting
      psl2d = np.matrix.transpose(psl2d)
      Unow, snow, VTnow = LA.svd(psl2d, full_matrices=False)
      century_trends[key]['U'] = Unow
      century_trends[key]['s'] = snow
      century_trends[key]['VT'] = VTnow
    for season in SEASONS:
      eigenpatterns['singVal 0']['anomalies'][season].append(century_trends[season]['VT'][0]-np.mean(century_trends[season]['VT'][0]))
      eigenpatterns['singVal 0']['normalized'][season].append(eigenpatterns['singVal 0']['anomalies'][season][member]/np.std(century_trends[season]['VT'][0]))  
    
      
    
  # for member in range(len(psl_ssp585.zstore.values)):
      nao_smoothed = moving_average(eigenpatterns['singVal 0']['anomalies'][season][member], 30) # Doing the moving average for 30 points because each season is 3 months per year - so this is a 10 year running mean
      eigenpatterns['singVal 0']['smoothed'][season].append(nao_smoothed/np.std(nao_smoothed))
      # Above, I took the running mean of the anomalies, then normalized. The following two lines will do this in reverse order. Why are the results different? Qualitatively they are the same, though, so I'm not worried in the short term.
      # nao_smoothed = moving_average(eigenpatterns['singVal 0']['normalized'][season][member], 30) # Doing the moving average for 30 points because each season is 3 months per year - so this is a 10 year running mean
      # eigenpatterns['singVal 0']['smoothed'][season].append(moving_average(eigenpatterns['singVal 0']['normalized'][season][member], 30))
    fig, axs = plt.subplots(2, 2, figsize=[24, 12])
    axlist = axs.flatten()
    for season in SEASONS:
      # x = np.arange(0, 495)
      x = np.arange(0, len(nao_smoothed))
      axlist[SEASONS.index(season)].fill_between(x, 0, eigenpatterns['singVal 0']['smoothed'][season][member], where=eigenpatterns['singVal 0']['smoothed'][season][member]>0, color='r', label='10 year running mean')
      axlist[SEASONS.index(season)].fill_between(x, 0, eigenpatterns['singVal 0']['smoothed'][season][member], where=eigenpatterns['singVal 0']['smoothed'][season][member]<0, color='b', label='10 year running mean')
      axlist[SEASONS.index(season)].plot(eigenpatterns['singVal 0']['normalized'][season][member], 'silver', zorder=0, label='Unsmoothed')
      # plt.plot(x)
      axlist[SEASONS.index(season)].set_title('NAO Index (PC 1) '+season)
      axlist[SEASONS.index(season)].set_ylabel('Right Singular Vector Normalized Anomalies')
      axlist[SEASONS.index(season)].set_xlabel('Time')
      axlist[SEASONS.index(season)].legend()
    plt.show()
smoothplot()